<a href="https://colab.research.google.com/github/la2015-hw/Group_10/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import files
files.upload()

Saving kaggle (1).json to kaggle (1).json


{'kaggle (1).json': b'{"username":"janyarathnakumar","key":"dd174795263eae7714baa51a7f0cd109"}'}

In [13]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [14]:
!kaggle datasets list

ref                                                                title                                                     size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------------------------------  --------------------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
shahzadi786/world-smartphone-market-2025                           World Smartphone Market 2025                             17795  2025-11-09 04:52:42.650000           1661         54  1.0              
sadiajavedd/students-academic-performance-dataset                  Students_Academic_Performance_Dataset                     8907  2025-10-23 04:16:35.563000           4157        103  1.0              
ayeshaimran123/social-media-and-mental-health-balance              Social Media and Mental Health Balance                    5941  2025-10-26 07:51:53.380000           5022         67  1.0

In [15]:
!kaggle datasets download -d vaishnavichintha/sdss-tabular-dataset

Dataset URL: https://www.kaggle.com/datasets/vaishnavichintha/sdss-tabular-dataset
License(s): unknown
  0% 0.00/10.7M [00:00<?, ?B/s]
100% 10.7M/10.7M [00:00<00:00, 1.28GB/s]


In [16]:
!unzip sdss-tabular-dataset.zip

Archive:  sdss-tabular-dataset.zip
  inflating: sdss_gz2_matched_cleaned_balanced.csv  


In [17]:
import pandas as pd
df = pd.read_csv("sdss_gz2_matched_cleaned_balanced.csv")
print(df.head())

             specobjid      dr8objid            dr7objid          ra  \
0   614788390251948032  1.237654e+18  587728665578700971  122.952644   
1  2433194008562919424  1.237665e+18  587739651567845579  231.428600   
2  2080728518915811328  1.237663e+18  587736975809773641  232.551060   
3  1563980851195176960  1.237662e+18  587736619321917717  234.050800   
4  3139079110530197504  1.237666e+18  587739843772940458  217.789000   

         dec     rastring    decstring    sample gz2class  \
0  42.216805  08:11:48.63  +42:13:00.5  original    SBc?m   
1  23.370867  15:25:42.86  +23:22:15.1  original      Ser   
2  27.134108  15:30:12.25  +27:08:02.8  original    SBc2m   
3  30.967339  15:36:12.19  +30:58:02.4  original    SBd2m   
4  20.449600  14:31:09.36  +20:26:58.6  original    SBa2l   

   total_classifications  ...  \
0                     47  ...   
1                     40  ...   
2                     44  ...   
3                     55  ...   
4                     34  ...   

 

In [18]:
x = df.drop(columns=["broad_class"])  # all features
y = df["broad_class"]                 # target (galaxy morphology)


In [19]:
df.select_dtypes(include=['object']).columns


Index(['rastring', 'decstring', 'sample', 'gz2class', 'broad_class'], dtype='object')

In [20]:
# Drop non-numeric or target columns
x = df.drop(columns=[
    "sample",
    "broad_class",    # target column
    "rastring", "decstring",      # coordinates
    "gz2class"        # text morphological shorthand
])

In [21]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(
    x_scaled, df["broad_class"], test_size=0.3, random_state=25, stratify=df["broad_class"]
)

print("Train shape:", x_train.shape, y_train.shape)
print("Test shape:", x_test.shape, y_test.shape)


Train shape: (33320, 229) (33320,)
Test shape: (14280, 229) (14280,)


In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

#hyperparameter
criterion = "entropy"
max_depth = 5
min_split = 2

dt = DecisionTreeClassifier(
    criterion=criterion,
    max_depth=max_depth,
    min_samples_split=min_split,
    random_state=25
)

dt.fit(x_train, y_train)

# Predictions
y_pred_train = dt.predict(x_train)
y_pred_test = dt.predict(x_test)

# Print results
print(f"Train Accuracy: {accuracy_score(y_train, y_pred_train):.4f}")
print(f"Test Accuracy : {accuracy_score(y_test, y_pred_test):.4f}")

print("\nClassification Report (Test Data):")
print(classification_report(y_test, y_pred_test))

print("\nConfusion Matrix (Test Data):")
print(confusion_matrix(y_test, y_pred_test))


Train Accuracy: 0.9887
Test Accuracy : 0.9860

Classification Report (Test Data):
                 precision    recall  f1-score   support

     Elliptical       0.97      0.98      0.98      3570
      Irregular       0.99      1.00      0.99      3570
  Spiral-barred       1.00      1.00      1.00      3570
Spiral-unbarred       0.98      0.97      0.98      3570

       accuracy                           0.99     14280
      macro avg       0.99      0.99      0.99     14280
   weighted avg       0.99      0.99      0.99     14280


Confusion Matrix (Test Data):
[[3494   20   11   45]
 [   5 3559    3    3]
 [   8    1 3553    8]
 [  84   12    0 3474]]
